# MASH
Summary of [MASH](https://github.com/marbl/Mash) results from project: `[{{ project().name }}]`

## Description
Fast genome and metagenome distance estimation using MinHash

In [ ]:
import pandas as pd
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

#import os
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.preprocessing import MinMaxScaler
import numpy as np
#import networkx as nx
#import community as community_louvain
sns.set_context("paper")


def kMeansRes(scaled_data, k, alpha_k=0.02):
    '''
    # Calculating clusters from https://medium.com/towards-data-science/an-approach-for-choosing-number-of-clusters-for-k-means-c28e614ecb2c
    Parameters 
    ----------
    scaled_data: matrix 
        scaled data. rows are samples and columns are features for clustering
    k: int
        current k for applying KMeans
    alpha_k: float
        manually tuned factor that gives penalty to the number of clusters
    Returns 
    -------
    scaled_inertia: float
        scaled inertia value for current k           
    '''
    
    inertia_o = np.square((scaled_data - scaled_data.mean(axis=0))).sum()
    # fit k-means
    kmeans = KMeans(n_clusters=k, random_state=0).fit(scaled_data)
    scaled_inertia = kmeans.inertia_ / inertia_o + alpha_k * k
    return scaled_inertia

def chooseBestKforKMeans(scaled_data, k_range):
    ans = []
    for k in k_range:
        scaled_inertia = kMeansRes(scaled_data, k)
        ans.append((k, scaled_inertia))
    results = pd.DataFrame(ans, columns = ['k','Scaled Inertia']).set_index('k')
    if len(results) > 0:
        best_k = results.idxmin()[0]
    else:
        print("WARNING: Cannot determine best k, returning k as 1.")
        best_k = 1
    return best_k, results

In [ ]:
report_dir = Path("../data/external/G1034_20230801/")

In [ ]:
df_mash = pd.read_csv(report_dir / 'mash/df_mash.csv', index_col=0)
#df_gtdb = pd.read_csv(report_dir / 'tables' / 'df_gtdb_meta.csv', index_col='genome_id')

## Hierarchical Clustering based on MASH distances

In [ ]:
df_mash_corr = df_mash.fillna(0).corr()

plt.figure(figsize=(30, 10))
#plt.title("MASH Distances", fontsize=20)  # You can adjust the title font size here

selected_data = df_mash_corr.copy()
clusters = shc.linkage(selected_data, 
            method='ward', 
            metric="euclidean",
            optimal_ordering=True)
shc.dendrogram(Z=clusters, labels=df_mash_corr.index, orientation='left')  # Set orientation to 'left'

plt.yticks(fontsize=14)  # Adjust the font size for y-axis ticks (labels) for horizontal dendrogram
plt.xticks(fontsize=14)  # Adjust the font size for y-axis ticks (labels) for horizontal dendrogram
plt.show()

## Estimate Number of Clusters

In [ ]:
# choose features
data_for_clustering = df_mash.copy()
data_for_clustering.fillna(0,inplace=True)

# create data matrix
data_matrix = np.asarray(data_for_clustering).astype(float)
data_matrix

# scale the data
mms = MinMaxScaler()
scaled_data = mms.fit_transform(data_matrix)

# choose k range
if len(df_mash) <= 21:
    max_range = len(df_mash) - 1
else:
    max_range = 20

k_range=range(2, max_range)
# compute adjusted intertia
best_k, results = chooseBestKforKMeans(scaled_data, k_range)

# plot the results
plt.figure(figsize=(7,4))
plt.plot(results,'o')
plt.title('Adjusted Inertia for each K')
plt.xlabel('K')
plt.ylabel('Adjusted Inertia')
plt.xticks(range(2,max_range,1))
print(f"Estimated number of clusters: {best_k}")
plt.show()

## MASH Clustermap

In [ ]:
n_clusters = best_k

# max color 12
if best_k < 12:
    top_clusters = best_k
else:
    top_clusters = 12

Agg_hc = AgglomerativeClustering(n_clusters = n_clusters, affinity = 'euclidean', linkage = 'ward')
y_hc = Agg_hc.fit_predict(df_mash_corr)
color_set3 = ['#8dd3c7','#ffffb3','#bebada','#fb8072','#80b1d3','#fdb462','#b3de69','#fccde5','#d9d9d9','#bc80bd','#ccebc5','#ffed6f']

df_hclusts = pd.DataFrame(index=df_mash_corr.index, columns=['hcluster', 'color_code'])
df_hclusts['hcluster'] = y_hc
top_clusters = df_hclusts.hcluster.value_counts().index.tolist()[:top_clusters]
dict_top_colors = dict(zip(top_clusters, color_set3[:len(top_clusters)]))

for genome_id in df_hclusts.index:
    cluster_id = df_hclusts.loc[genome_id, 'hcluster']
    if cluster_id in top_clusters:
        df_hclusts.loc[genome_id, 'color_code'] = dict_top_colors[cluster_id]
    else:
        df_hclusts.loc[genome_id, 'color_code'] = "#808080"
        
comm_colors = df_hclusts['color_code']
plt.figure()


# sns.set_theme(color_codes=True)
g = sns.clustermap((1 - df_mash)*100,
                  figsize=(20,20), row_linkage=clusters, col_linkage=clusters,
                  row_colors=comm_colors, col_colors=comm_colors, cmap="rocket_r")
g.ax_heatmap.set_xlabel('Genomes', fontsize=18)
g.ax_heatmap.set_ylabel('Genomes', fontsize=18)
plt.setp(g.ax_heatmap.get_xticklabels(), rotation=90, fontsize=12)  # set rotation and font size for x-axis labels
plt.setp(g.ax_heatmap.get_yticklabels(), rotation=0, fontsize=12)  # set font size for y-axis labels

# Adjust font size of the colorbar's tick labels
cbar = g.cax
cbar.set_yticklabels(cbar.get_yticklabels(), fontsize=16)

plt.show()

In [ ]:
df_mash_corr

In [ ]:
import numpy as np
import networkx as nx
import community as community_louvain
import matplotlib.pyplot as plt

# Assuming `dist_matrix` is your MASH distance matrix
similarity_matrix = 1 - df_mash

# Create a graph from the similarity matrix
G = nx.from_pandas_adjacency(similarity_matrix)

# Remove edges with weight <= 0.95
edges_to_remove = [(u, v) for u, v, data in G.edges(data=True) if data['weight'] <= 0.95]
G.remove_edges_from(edges_to_remove)

# Use the Louvain method to detect communities
partition = community_louvain.best_partition(G)

# `partition` is a dictionary where the keys are the nodes and the values are the community IDs
pos = nx.spring_layout(G)
nx.draw_networkx_nodes(G, pos, cmap=plt.get_cmap('jet'), node_size = 20)
nx.draw_networkx_edges(G, pos, alpha=0.5)
plt.show()

In [ ]:
df_louvainn = pd.DataFrame.from_dict({"Louvainn_partition" : partition})
df_louvainn["Louvainn_partition"].value_counts()

In [ ]:
self_links = [(n, n) for n in G.nodes() if G.has_edge(n, n)]
G.remove_edges_from(self_links)
pos = nx.spring_layout(G)
nx.draw_networkx_nodes(G, pos, cmap=plt.get_cmap('jet'), node_size = 20)
nx.draw_networkx_edges(G, pos, alpha=0.5)
plt.show()

In [ ]:
# Use the Louvain method to detect communities
partition = community_louvain.best_partition(G)
df_louvainn = pd.DataFrame.from_dict({"Louvainn_partition" : partition})
df_louvainn["Louvainn_partition"].value_counts()

## References
<font size="2">
{% for i in project().rule_used['mash']['references'] %}
- *{{ i }}*
{% endfor %}
</font>